In [30]:
import pandas as pd
import numpy as np
from typing import List, Tuple

In [31]:
#### Parameters
spot_filename:str = "/tmp/index.csv"
options_filename:str = "../../data/nifty_options_eod.h5"
output_filename:str = 'output.csv' # Friendly name for output
start_day:int = 4
holding_days:int = 7
de:int = 0
step:int = 100 
options:Tuple[Tuple[str, float]] = (
    ('p', 0),
    ('p', 0.01),
    ('p', 0.02),
    ('p', 0.03),
    ('c', 0.0),
    ('c', 0.01),
    ('c', 0.02),
    ('c', 0.03)
)


In [32]:
index = pd.read_csv(spot_filename, parse_dates=['date'])
opt = pd.read_hdf(options_filename).sort_values(by=['date'])
rename = {
    'open_price': 'open',
    'high_price': 'high',
    'low_price': 'low',
    'close_pric': 'close'
}
opt = opt.rename(columns=rename)
opt['de'] = (opt['expiry_date'] - opt['date']).dt.days

In [33]:
ce = opt.query("opt_type=='CE'")
pe = opt.query("opt_type=='PE'")

In [34]:
def get_result(options_data, spot_data, opt='p', wkday=4, de=0, itm=0, step=100, holding_days=7):
    weekly = spot_data.query(f"wkday=={wkday}").copy()
    if opt.lower() == 'p':
        m = 1-itm
    else:
        m = 1+itm
    weekly['strike'] = [int((x*m)/step)*step for x in weekly.close.values]
    if opt.lower() == 'p':
        weekly['strike'] = weekly['strike'] + step
    opts = options_data.merge(weekly[['date', 'close', 'strike']].rename(
        columns={'close':'spot'}), on=['date', 'strike'])
    opts = opts.query(f"de>{de}")
    opts = opts.sort_values(by='de').groupby('date').first().reset_index()
    cols = ['date', 'contract_d', 'strike', 'expiry_date', 'close', 'spot']
    entries = opts[cols].copy()
    entries['entry_date'] = entries.date.copy()
    exit_frame = entries.copy()
    exit_frame['date'] = exit_frame.date + pd.DateOffset(days=holding_days)
    del exit_frame['close'] # Since we need the other close in expiry
    del exit_frame['contract_d'] 
    del exit_frame['spot']
    opt2 = exit_frame.merge(options_data, on=['date', 'strike', 'expiry_date'])
    opt2 = opt2.merge(index[['date', 'close']].rename(
        columns={'close': 'spot'}), on=['date'])
    exits = opt2[cols + ['entry_date']]
    exits = exits.rename(columns={'date':'exit_date', 'entry_date': 'date'})
    trades = entries.merge(exits, on=['strike', 'expiry_date', 'contract_d', 'date'])
    rename = {
        'date_x': 'entry_date',
        'close_x': 'entry_price',
        'spot_x': 'entry_spot',
        'date_y': 'exit_date',
        'close_y': 'exit_price',
        'spot_y': 'exit_spot'
    }
    trades = trades.rename(columns=rename)
    trades['pnl'] = trades.eval('exit_price-entry_price')
    trades['de'] = (trades['expiry_date'] - trades['entry_date']).dt.days
    return trades


In [35]:
collect = []
for i,(opt,strike) in enumerate(options):
    data = ce if opt =='c' else pe
    temp = get_result(data, index, itm=strike,step=step, wkday=start_day, 
                      holding_days=holding_days, opt=opt)
    temp['name'] = f"opt{i}"
    collect.append(temp)
res = pd.concat(collect)
del collect
res.pnl.describe()

count    2056.000000
mean       -0.483512
std       133.569121
min      -496.150000
25%       -54.475000
50%       -19.375000
75%        13.300000
max      1564.200000
Name: pnl, dtype: float64

In [36]:
res2 = res.query('de<=7').pivot(index='date', columns=['name'], values='pnl')
res2.loc['2019':].describe()

name,opt0,opt1,opt2,opt3,opt4,opt5,opt6,opt7
count,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000
mean,-20.753846,-12.967521,-8.405983,-2.690171,20.591453,19.090171,12.663675,8.123932
std,244.366807,215.898997,188.499834,170.891073,202.549330,167.156925,132.001719,102.619519
min,-496.150000,-451.950000,-403.850000,-403.850000,-352.050000,-296.250000,-244.450000,-198.500000
25%,-134.750000,-90.700000,-61.650000,-38.250000,-126.200000,-78.900000,-43.050000,-19.800000
50%,-87.650000,-55.700000,-35.750000,-19.950000,-34.400000,-46.700000,-19.950000,-9.650000
75%,19.900000,-27.550000,-19.500000,-9.200000,144.700000,75.750000,-3.050000,-3.950000
max,1564.200000,1504.800000,1441.200000,1370.450000,864.800000,811.050000,752.450000,613.900000


In [37]:
res.to_csv('/tmp/output2.csv')

In [38]:
res.to_csv(output_filename)